# Data Validation

In this notebook, we are going to take a peek at the outcome of the Item 7 parse for all companies. Let's see if we can gain some insights on how successful the pull was:

In [50]:
import numpy as np
import pandas as pd
import pickle
import os
import re
import pdfkit
from tika import parser
from pdf2image import convert_from_path, convert_from_bytes

In [60]:
with open('../Step1-Data/4-link_dict.pickle', 'rb') as f:
    link_dict = pickle.load(f)

In [61]:
link_dict.keys()

dict_keys(['0000006201', '0001158449', '0000320193', '0001551152', '0001140859', '0000077551', '0000874501', '0000815094', '0000003333', '0000001800', '0000817473', '0001109304', '0001467373', '0000002135', '0001368457', '0000796343', '0000061478', '0000006281', '0000007084', '0000008670', '0001101215', '0000769397', '0001546640', '0001002910', '0000004904', '0000874761', '0001122304', '0000004977', '0001391461', '0000850693', '0000005272', '0000109563', '0000922864', '0001267238', '0000354190', '0001086222', '0000918160', '0001487712', '0000915913', '0001097149', '0000766421', '0000899051', '0001579241', '0000768251', '0000899866', '0000005133', '0000006951', '0000711065', '0000002488', '0001037868', '0001004434', '0000318154', '0001562401', '0000820027', '0001053507', '0001018724', '0000350698', '0000317093', '0001018840', '0001013462', '0001376610', '0000315293', '0000091142', '0000006769', '0000773910', '0000835910', '0000002969', '0000820313', '0000929887', '0001433270', '00013051

In [62]:
companies = list(link_dict.keys())
link_dict[companies[0]]

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,Link,Loc7,Loc8
49,0000006201-21-000014,2021-02-17,2020-12-31,2021-02-17T17:17:57.000Z,34,10-K,001-08400,21646186,,43925703,1,1,aal-20201231.htm,10-K 2020 02.17.21,https://www.sec.gov/Archives/edgar/data/6201/0...,"[[(158292, 158310), Item 7. Management], [(167...","[[(502641, 502662), ITEM 8A. CONSOLIDATED]]"
150,0000006201-20-000023,2020-02-19,2019-12-31,2020-02-19T07:31:30.000Z,34,10-K,001-08400,20627428,,30851334,1,1,a10k123119.htm,10-K 2019 02.19.20,https://www.sec.gov/Archives/edgar/data/6201/0...,"[[(153128, 153146), Item 7. Management], [(156...","[[(414897, 414918), ITEM 8A. CONSOLIDATED]]"
225,0000006201-19-000009,2019-02-25,2018-12-31,2019-02-25T07:31:34.000Z,34,10-K,001-08400,19628071,,30572408,1,0,a10k123118.htm,10-K 2018 02.25.19,https://www.sec.gov/Archives/edgar/data/6201/0...,"[[(9505, 9523), Item 7. Management], [(12796, ...","[[(300867, 300888), ITEM 8A. CONSOLIDATED]]"
315,0000006201-18-000009,2018-02-21,2017-12-31,2018-02-21T08:02:40.000Z,34,10-K,001-08400,18627088,,27914491,1,0,a10k123117.htm,10-K,https://www.sec.gov/Archives/edgar/data/6201/0...,"[[(9554, 9572), Item 7. Management], [(13606, ...","[[(293380, 293401), ITEM 8A. CONSOLIDATED]]"
412,0001193125-17-051216,2017-02-22,2016-12-31,2017-02-22T08:01:43.000Z,34,10-K,001-08400,17627073,,24888480,1,0,d286458d10k.htm,FORM 10-K,https://www.sec.gov/Archives/edgar/data/6201/0...,"[[(9935, 9953), Item 7. Management], [(14047, ...","[[(297249, 297270), ITEM 8A. CONSOLIDATED]]"
538,0001193125-16-474605,2016-02-24,2015-12-31,2016-02-24T08:04:10.000Z,34,10-K,001-08400,161450518,,26170400,1,0,d78287d10k.htm,FORM 10-K,https://www.sec.gov/Archives/edgar/data/6201/0...,"[[(17027, 17045), Item 7. Management], [(21453...","[[(398001, 398022), ITEM 8A. CONSOLIDATED]]"
651,0001193125-15-061145,2015-02-25,2014-12-31,2015-02-25T08:02:34.000Z,34,10-K,001-08400,15645918,,39524925,1,0,d829913d10k.htm,FORM 10-K,https://www.sec.gov/Archives/edgar/data/6201/0...,"[[(16174, 16192), Item 7. Management], [(23008...","[[(452689, 452710), ITEM 8A. CONSOLIDATED]]"
750,0000006201-14-000004,2014-02-28,2013-12-31,2014-02-28T07:52:16.000Z,34,10-K,001-08400,14651496,,47888955,1,0,aagaa10k-20131231.htm,10-K,https://www.sec.gov/Archives/edgar/data/6201/0...,"[[(15590, 15608), Item 7. Management], [(23363...",[]


In [63]:
[len(x) for x in link_dict[companies[0]]['Item7'].values]

KeyError: 'Item7'

In [12]:
aggregated_lens = [len(x) for i in range(len(companies)) for x in link_dict[companies[i]]['Item7'].values]

In [16]:
aggregated_lens.count(0)

4594

In [17]:
len(aggregated_lens)

8052

In [20]:
badslist = []
for company in companies:
    bads = [len(x) for x in link_dict[company]['Item7'].values]
    if bads.count(0) > 0:
        badslist.append(company)

In [49]:
len(badslist)

575

In [27]:
link = link_dict[badslist[0]]['Link'].values[0]

In [31]:
pdfkit.from_url(link, 'out.pdf')

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                        


True

In [36]:
rawText = parser.from_file('out.pdf')
rawList = rawText['content'].splitlines()
result = [x for x in rawList if x[:4] == 'ITEM']
result

[]

In [43]:
result = [x for x in rawList if re.match('ITEM', x[:4], re.IGNORECASE)]

In [45]:
matches = re.compile(r'^ITEM.*7[^A][^A]', re.IGNORECASE)
item7 = [matches.search(x) for x in result]
idx7 = rawList.index([x for x in rawList if matches.search(x) != None][-1])

##### ITEM 8
matches = re.compile(r'^ITEM.*8[^B][^B]', re.IGNORECASE)
item7 = [matches.search(x) for x in result]
idx8 = rawList.index([x for x in rawList if matches.search(x) != None][-1])

43600